## Function Calling

In [28]:
from helper import Document
with open('./prep/documents.json', 'r') as file:
            json_documents = json.load(file)

loaded_documents = [Document.from_json(json_str) for json_str in json_documents]

In [186]:
from datetime import datetime

def filter_events_by_day_and_time_constraints(events, target_day, earliest_start_time="00:00", latest_end_time=None, cutoff_start_time="23:59"):
    earliest_start_time_obj = datetime.strptime(earliest_start_time, "%H:%M").time()
    latest_end_time_obj = None if latest_end_time is None else datetime.strptime(latest_end_time, "%H:%M").time()
    cutoff_start_time_obj = datetime.strptime(cutoff_start_time, "%H:%M").time()

    filtered_events = []
    for event in events:
        try:
            event_day = event.metadata['date']['day']
            event_start_time_str = event.metadata['date']['start_time']
            event_end_time_str = event.metadata['date'].get('end_time')  # Handle case where end_time might not be provided
            
            if event_day == target_day and event_start_time_str:
                event_start_time = datetime.strptime(event_start_time_str, "%H:%M").time()
                
                # Filter events that start before the specified cutoff time
                if cutoff_start_time_obj and event_start_time < cutoff_start_time_obj:
                    filtered_events.append(event)
                elif earliest_start_time_obj and event_start_time >= earliest_start_time_obj:
                    if latest_end_time_obj and event_end_time_str:
                        event_end_time = datetime.strptime(event_end_time_str, "%H:%M").time()
                        if event_end_time <= latest_end_time_obj:
                            filtered_events.append(event)
                    else:
                        filtered_events.append(event)
        except ValueError:
            continue  # Skip this event if there's a format error

    return filtered_events


In [188]:
from dotenv import load_dotenv

load_dotenv()  # This loads the environment variables from .env

True

In [189]:
from openai import OpenAI
client = OpenAI()
GPT_MODEL = "gpt-3.5-turbo"
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e



In [190]:
tools = [{
    "type": "function",
    "function": {
        "name": "filter_events_by_day_and_time_constraints",
        "description": "Filters a list of events based on specific day and time constraints, including earliest and latest permissible times for event start and end.",
        "parameters": {
            "type": "object",
            "properties": {
                "target_day": {
                    "type": "string",
                    "description": "The specific day of the week to filter events for, e.g., 'Monday', 'Tuesday'."
                },
                "earliest_start_time": {
                    "type": "string",
                    "description": "The earliest permissible start time for events, in 24-hour format, e.g., '08:00' for 8 AM."
                },
                "latest_end_time": {
                    "type": "string",
                    "description": "The latest permissible ending time for events, if specified, in 24-hour format, e.g., '17:00' for 5 PM."
                },
                "cutoff_start_time": {
                    "type": "string",
                    "description": "The cutoff start time for considering events, any event starting after this time will be excluded, e.g., '23:00'."
                }
            }
        }
    }
}]


In [180]:
import json
def execute_function_call(message):
    if message.tool_calls[0].function.name == "filter_events_by_day_and_time_constraints":
        kwargs = json.loads(message.tool_calls[0].function.arguments)
        results = filter_events_by_day_and_time_constraints(loaded_documents, **kwargs)
    else:
        results = f"Error: function {message.tool_calls[0].function.name} does not exist"
    return results

In [184]:
messages = []
messages.append({"role": "system", "content": waterkant_festival_description+"You are a helpful chatbot for the attendees of the Waterkant Festival! You answer questions about events, schedules, venue information, transportation options, or anything else. The attendees should feel free to ask you. You respond in the same language the user asks the question!"})
messages.append({"role": "user", "content": "Seit wann gibt es das Waterkant festival"})
chat_response = chat_completion_request(messages, tools)
assistant_message = chat_response.choices[0].message

if assistant_message.tool_calls:
    assistant_message.content = str(assistant_message.tool_calls[0].function)
    messages.append({"role": assistant_message.role, "content": assistant_message.content})
    results = execute_function_call(assistant_message)
    messages.append({"role": "function", "tool_call_id": assistant_message.tool_calls[0].id, "name": assistant_message.tool_calls[0].function.name, "content": results})
else:
    messages.append({"role": assistant_message.role, "content": assistant_message.content})

In [185]:
messages

[{'role': 'system',
  'content': "The Waterkant Festival is a dynamic and innovative event that takes place annually in Kiel, Germany. Since its inception in 2016, the festival has served as a platform for startups, companies, and creative thinkers to showcase and discuss their ideas on technology, sustainability, and future societal structures. The festival is organized by opencampus.sh, a non-profit association aimed at fostering education and entrepreneurship in Schleswig-Holstein.\n\nTypically held over two days, Waterkant Festival features a diverse array of activities including talks, workshops, and interactive exhibitions. Each year, it gathers hundreds of participants and speakers from various fields—ranging from business and science to politics and the arts—to engage in future-oriented discussions and collaborations. The festival's format includes themed sessions that cover a wide range of topics such as artificial intelligence, sustainability, new work methods, and innovative

In [17]:
json.loads(assistant_message.tool_calls[0].function.arguments)

{'day': 'Friday', 'min_start_time': '00:00'}

In [20]:
assistant_message.tool_calls[0].function.name

'filter_events_by_day_and_time'

In [11]:
from datetime import datetime

# Get the current datetime
now = datetime.now()

# Extract day name, hour, and minute
day_name = now.strftime("%A")
start_time = now.strftime("%H:%M")

# Create a dictionary with formatted output
formatted_output = {
    'day': day_name,
    'current_time': start_time
}

# Print results
print(formatted_output)


{'day': 'Saturday', 'current_time': '18:22'}


In [124]:
template="""Extract time-related information such as days of the week, start times, and end times from a question, and format them as specified, follow these steps:

Identify Time Information: Begin by scrutinizing the question to identify mentions of specific days, times, or durations. Key elements to look for include:
Days of the week (e.g., Monday, Tuesday, etc.).
Time expressions (e.g., "9 AM", "1400 hours", "from 9 to 11").
Keywords suggesting time durations (e.g., "morning", "evening", "all day").
List Days of the Week: If any days are mentioned:
Note each day explicitly mentioned. Include all days in a mentioned range (e.g., "Monday to Friday").
Ensure days are fully spelled out (e.g., "Monday", not "Mon") and capitalized.
Determine Start and End Times:
Look for specific times mentioned for starting or ending, noted in either 12-hour or 24-hour format.
Decide if each time refers to a start time or end time based on contextual keywords like "from", "start at", "until", "end by".
Convert times in 12-hour format (AM/PM) into 24-hour format for consistency (e.g., "2 PM" becomes "14:00").
Calculate Minimum Start Time and Maximum End Time:
From all start times identified, choose the earliest as "min_start_time".
From all end times, select the latest as "max_end_time".
Times should be in the "HH:MM" format (e.g., "09:00", "18:00").
Check for Default Values:
If no days are mentioned in the question, default to ['Thursday', 'Friday'].
If no start or end times are provided, use '00:00' as the default "min_start_time" and '23:00' as the "max_end_time".
Format the Output:
Organize the data into a dictionary format with keys: 'days', 'min_start_time', and 'max_end_time'.
Under 'days', include the listed or default days in square brackets.
Assign the respective or default values to 'min_start_time' and 'max_end_time'.
Example:
If the question is "When are late night discussions held?", and no specific days or times are mentioned, your output should look like this:

{
  "days": ["Thursday", "Friday"],
  "min_start_time": "00:00",
  "max_end_time": "23:00"
}
Example:
If the question is "What are the events on Friday starting at 9 AM and ending before 4 PM?", your formatted output should look like this:
{
  "days": ["Friday"],
  "min_start_time": "00:00",
  "max_end_time": "16:00"
}

Here is the question:
%question%
"""

In [125]:
def replace_placeholders(template, replacements):
    for key, value in replacements.items():
        placeholder = "%" + key + "%"
        template = template.replace(placeholder, value)
    return template

In [135]:
question = "What events are about on Friday at 7 pm on sustainability"

In [136]:
replacements= {"question": question}

In [137]:
prompt= replace_placeholders(template, replacements)

In [138]:
messages= [{"role": "user", "content": prompt}]

In [139]:
response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.0)

In [140]:
response.choices[0].message.content

'{\n  "days": ["Friday"],\n  "min_start_time": "19:00",\n  "max_end_time": "19:00"\n}'

In [141]:
import json

In [142]:
json.loads(str(response.choices[0].message.content))

{'days': ['Friday'], 'min_start_time': '19:00', 'max_end_time': '19:00'}

In [169]:
question = "Welche KI Events sind am Freitag bis 15 Uhr"

In [170]:
waterkant_festival_description = """The Waterkant Festival is a dynamic and innovative event that takes place annually in Kiel, Germany. Since its inception in 2016, the festival has served as a platform for startups, companies, and creative thinkers to showcase and discuss their ideas on technology, sustainability, and future societal structures. The festival is organized by opencampus.sh, a non-profit association aimed at fostering education and entrepreneurship in Schleswig-Holstein.

Typically held over two days, Waterkant Festival features a diverse array of activities including talks, workshops, and interactive exhibitions. Each year, it gathers hundreds of participants and speakers from various fields—ranging from business and science to politics and the arts—to engage in future-oriented discussions and collaborations. The festival's format includes themed sessions that cover a wide range of topics such as artificial intelligence, sustainability, new work methods, and innovative technology applications.

Waterkant also provides a stage for startups to pitch their ideas and for thought leaders to share insights. The event fosters a strong sense of community and collaboration, with numerous opportunities for networking and co-creation. In addition to professional interactions, the festival incorporates elements of cultural engagement, including live music and social gatherings, creating a vibrant festival atmosphere directly by the waterfront.

Waterkant is notable for its commitment to innovation and its role in promoting the economic and educational development of the region. It supports budding entrepreneurs and established businesses alike, providing a space to explore new ideas and push the boundaries of conventional industry practices​"""

In [171]:
template="General info\n\n"+waterkant_festival_description+"\n--------------\n"+"Answer the user question in the language of the user if the question is answered by the text above. If the question aims at something that has nothing to do, inform the user that he should only ask questions regarding the topis around the Waterkant Festival. If the user asks about activities and events return exactly this: 'events'.\n Example:\nQuestion:\n What is the difference between Burger and Pommes?\nPlease ask only questions regardings the Waterkant festival. Thanks!\n\n Here is the question:\n%question%"

In [172]:
system_prompt = "You are a helpful chatbot for the attendees of the Waterkant Festival! You answer questions about events, schedules, venue information, transportation options, or anything else. The attendees should feel free to ask you. You respond in the same language the user asks the question!"

In [173]:
replacements= {"question": question}

In [174]:
prompt= replace_placeholders(template, replacements)

In [175]:
prompt

"General info\n\nThe Waterkant Festival is a dynamic and innovative event that takes place annually in Kiel, Germany. Since its inception in 2016, the festival has served as a platform for startups, companies, and creative thinkers to showcase and discuss their ideas on technology, sustainability, and future societal structures. The festival is organized by opencampus.sh, a non-profit association aimed at fostering education and entrepreneurship in Schleswig-Holstein.\n\nTypically held over two days, Waterkant Festival features a diverse array of activities including talks, workshops, and interactive exhibitions. Each year, it gathers hundreds of participants and speakers from various fields—ranging from business and science to politics and the arts—to engage in future-oriented discussions and collaborations. The festival's format includes themed sessions that cover a wide range of topics such as artificial intelligence, sustainability, new work methods, and innovative technology appli

In [176]:
from openai import OpenAI
client = OpenAI()
messages=[
        {"role": "system", "content": system_prompt},
      ]
messages+= [{"role": "user", "content": prompt}]
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=0.0
)

print(completion.choices[0].message.content)

Am Freitag bis 15 Uhr finden verschiedene KI-Events auf dem Waterkant Festival statt. Dazu gehören Vorträge, Workshops und interaktive Ausstellungen, die sich mit künstlicher Intelligenz und anderen zukunftsorientierten Themen befassen.
